# Import Useful Modules 

In [1]:
# Script untuk membatasi konsumsi gpu memory(kalau ditrain pakai AI-1)

import tensorflow
config = tensorflow.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5 # isi dengan fraction memory yang ingin dipakai (maksimal 12 GB)
session = tensorflow.Session(config=config)

D:\Program\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import gmtime, strftime
import time
import datetime
from collections import Counter
import pickle
import os
import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Preprocessing
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Embedder
from gensim.models import FastText

# Classifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.linear_model import LogisticRegression as LR
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.grid_search import GridSearchCV as GS
from xgboost import XGBClassifier as XGB
from sklearn.decomposition import PCA,TruncatedSVD
from sklearn.preprocessing import OneHotEncoder as OHE
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.utils import to_categorical
from keras import regularizers

from preprocessing_pipeline import preprocessing

D:\Program\Anaconda\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
D:\Program\Anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
D:\Program\Anaconda\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
D:\Program\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversi

In [3]:
# Menentukan folder yang akan digunakan untuk menyimpan model. Saya sengaja pakai banyak folder agar model yang
# berbeda tidak tercampur. File saling terkait satu sama lain, bisa kacau kalau tercampur.
version="version_x"

if(not(os.path.exists("model/{}".format(version)))):
    os.makedirs("model/{}".format(version))

In [4]:
# Import kelas preprocessing(buatan saya sendiri, file jangan lupa diletakkan di folder yang sama)
from preprocessing_pipeline import preprocessing

# Import object fasttext
with open("model/{}/word_embedder.pickle".format(version), "rb") as file:
    word_embedder = pickle.load(file)
    
preprocessor=preprocessing(word_embedder.vector_size,word_embedder)
preprocessor

# ---------------------------------------------------------------------------------------------------------------

# Read Data To Pandas Dataframe

In [5]:
# Membaca data besar (+- 2000k datapoints, +-230 classes)
# Output berupa 2 list of labels(features dan labels)

def read_product_title_data():
    product_title_only=pd.read_csv('data/query.csv',header=None)
    product_title_only.dropna(inplace=True,axis=0)
    features=product_title_only[1]
    labels=product_title_only[0]
    
    return features,labels

In [6]:
# Membaca data kecil (+-400k datapoints, +-110 classes)
# Output berupa 2 list of labels(features dan labels)

def read_product_title_data_small():
    product_title_only=pd.read_csv('data/big.csv',header=None)
    product_title_only.dropna(inplace=True,axis=0)
    features=product_title_only[1]
    labels=product_title_only[0]
      
    return features,labels

In [7]:
data_for_embedding=read_product_title_data_small()

In [8]:
nfnf,nlnl=preprocessor.getFilteredClasses(data_for_embedding[0],data_for_embedding[1],110)

1/3
2/3


396099it [00:00, 528843.80it/s]


#  ---------------------------------------------------------------------------------------------------

In [9]:
product_title=[preprocessor.tokenize(value) for value in tqdm.tqdm(nfnf)]

100%|█████████████████████████████████████████████████████████████████████████| 396099/396099 [00:04<00:00, 88534.79it/s]


In [10]:
new_data=preprocessor.getFilteredData(product_title,list(nlnl),50,2,3)

1/1


100%|████████████████████████████████████████████████████████████████████████| 396099/396099 [00:00<00:00, 490112.15it/s]
396099it [00:00, 1599386.59it/s]


In [12]:
fff=new_data[0]
lll=new_data[1]

top_3={}
for label in tqdm.tqdm(set(lll)):
    top_3[label]=[value[0] for value in sorted(preprocessor.word_count_label(fff,lll,label).items(), key=lambda x: x[1])[-30:]]
    
allw=[]
for label in list(top_3):
    for i in top_3[label]:
        allw.append(i)
        

  0%|                                                                                            | 0/110 [00:00<?, ?it/s]

1/1
1/1
1/1


  3%|██▎                                                                                 | 3/110 [00:00<00:03, 27.70it/s]

1/1
1/1
1/1
1/1


  6%|█████▎                                                                              | 7/110 [00:00<00:03, 29.02it/s]

...

 95%|█████████████████████████████████████████████████████████████████████████████▌    | 104/110 [00:03<00:00, 30.16it/s]

1/1
1/1
1/1
1/1


 98%|████████████████████████████████████████████████████████████████████████████████▌ | 108/110 [00:03<00:00, 30.38it/s]

1/1
1/1


100%|██████████████████████████████████████████████████████████████████████████████████| 110/110 [00:03<00:00, 30.44it/s]


In [13]:
aaaaa=preprocessor.getTfIdfCustom(new_data[0],set(allw))

1/3


100%|███████████████████████████████████████████████████████████████████████| 233661/233661 [00:00<00:00, 1574187.83it/s]


2/3
3/3


In [14]:
newnew=[]
for index,value in tqdm.tqdm(enumerate(aaaaa[0].toarray())):
    temp=list(value)+list(preprocessor.vectorize_sentence(new_data[0][index]))
    newnew.append(temp) 

0it [00:00, ?it/s]


TypeError: 'NoneType' object is not iterable

In [ ]:
llee=LabelEncoder()
wenwen=llee.fit_transform(lll)

# ---------------------------------------------------------------------------------------------------------------

In [40]:
nn_X_train,nn_X_test,nn_y_train,nn_y_test=train_test_split(newnew,to_categorical(wenwen),test_size=0.1)

In [43]:
model = Sequential()
model.add(Dense(1000, input_shape=(len(nn_X_train[0]),), activation='relu'))
model.add(Dense(750, activation='relu'))
model.add(Dense(500, activation='relu'))
model.add(Dense(len(nn_y_train[0]), activation='softmax'))


model.compile(optimizer="Adagrad", loss='categorical_crossentropy',metrics=['accuracy'])

In [44]:
history = model.fit([nn_X_train], nn_y_train, initial_epoch=0, epochs=25, batch_size=100, validation_data=([nn_X_test],nn_y_test), shuffle=True)

Train on 927870 samples, validate on 103097 samples
Epoch 1/25
927870/927870 [==============================] - 34s 37us/step - loss: 0.5808 - acc: 0.8321 - val_loss: 0.5025 - val_acc: 0.8504
Epoch 2/25
927870/927870 [==============================] - 33s 36us/step - loss: 0.4546 - acc: 0.8620 - val_loss: 0.4589 - val_acc: 0.8611
Epoch 3/25
927870/927870 [==============================] - 34s 37us/step - loss: 0.4090 - acc: 0.8725 - val_loss: 0.4363 - val_acc: 0.8668
Epoch 4/25
927870/927870 [==============================] - 34s 37us/step - loss: 0.3795 - acc: 0.8789 - val_loss: 0.4222 - val_acc: 0.8706
Epoch 5/25
927870/927870 [==============================] - 40s 43us/step - loss: 0.3588 - acc: 0.8835 - val_loss: 0.4101 - val_acc: 0.8724
Epoch 6/25
927870/927870 [==============================] - 43s 47us/step - loss: 0.3426 - acc: 0.8871 - val_loss: 0.4022 - val_acc: 0.8740
Epoch 7/25
927870/927870 [==============================] - 43s 46us/step - loss: 0.3299 - acc: 0.8901 - val

KeyboardInterrupt: 

In [39]:
pred=model.predict(nn_X_test)

KeyboardInterrupt: 

In [55]:
print(classification_report(model.predict_classes([nn_X_test]),[np.argmax(value) for value in nn_y_test]))

             precision    recall  f1-score   support

          0       0.52      0.65      0.58       302
          1       0.04      0.83      0.08         6
          2       0.73      0.54      0.62      7291
          3       0.78      0.68      0.72      8984
          4       0.34      0.35      0.35       420
          5       0.72      0.57      0.63      7886
          6       0.67      0.61      0.64      8456
          7       0.45      0.47      0.46       535
          8       0.21      0.50      0.30       383
          9       0.46      0.54      0.50       397
         10       0.51      0.55      0.53       800
         11       0.43      0.48      0.45       658
         12       0.58      0.60      0.59       742
         13       0.35      0.45      0.40       444
         14       0.70      0.65      0.68      1182
         15       0.08      1.00      0.15         4
         16       0.81      0.86      0.83       527
         17       0.53      0.47      0.50   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


# ---------------------------------------------------------------------------------------------------------------

In [18]:
#preprocess product title to 100-dimensional vector
#and preprocess category name to integer label
large_embedded_data, large_label_encoder = preprocessor.preprocess_data(
    nfnf,
    nlnl,
)

  1%|          | 4192/396099 [00:00<00:09, 41905.65it/s]

REMOVING UNIMPORTANT CHARACTERS


100%|██████████| 396099/396099 [00:05<00:00, 76953.19it/s]


APPLYING FILTER


100%|██████████| 396099/396099 [00:01<00:00, 253086.06it/s]
396099it [00:00, 1277598.43it/s]
100%|██████████| 2495/2495 [01:33<00:00, 26.75it/s]


...

  0%|          | 550/233661 [00:00<01:24, 2743.54it/s]

CONVERTING SENTENCE TO VECTOR


  0%|          | 1/300 [00:00<00:37,  8.06it/s]

SAVE VECTOR TO PANDAS DATAFRAME


100%|██████████| 300/300 [00:41<00:00,  7.31it/s]


In [19]:
large_embedded_data.head()
large_embedded_data["sum"]=large_embedded_data.drop(["Labels"],axis=1).sum(axis=1)
large_embedded_data=large_embedded_data.loc[large_embedded_data["sum"]!=0].drop("sum",axis=1)

In [20]:
nn_X_train,nn_X_test,nn_y_train,nn_y_test=train_test_split(large_embedded_data.drop("Labels",axis=1),to_categorical(large_embedded_data["Labels"]),test_size=0.1)

In [50]:
model = Sequential()
model.add(Dense(600, input_shape=(300,), activation='relu'))
model.add(Dense(400, activation='relu'))
model.add(Dense(800, activation='relu'))
model.add(Dense(110, activation='softmax'))


model.compile(optimizer="Adagrad", loss='categorical_crossentropy',metrics=['accuracy','top_k_categorical_accuracy'])

In [51]:
history = model.fit(nn_X_train, nn_y_train, initial_epoch=0, epochs=30, batch_size=100, validation_data=(nn_X_test,nn_y_test), shuffle=True)

Train on 210012 samples, validate on 23335 samples
Epoch 1/30
210012/210012 [==============================] - 10s 46us/step - loss: 1.1154 - acc: 0.7109 - top_k_categorical_accuracy: 0.9264 - val_loss: 0.9243 - val_acc: 0.7416 - val_top_k_categorical_accuracy: 0.9511
Epoch 2/30
210012/210012 [==============================] - 8s 39us/step - loss: 0.8205 - acc: 0.7693 - top_k_categorical_accuracy: 0.9603 - val_loss: 0.8528 - val_acc: 0.7627 - val_top_k_categorical_accuracy: 0.9562
Epoch 3/30
210012/210012 [==============================] - 8s 40us/step - loss: 0.7446 - acc: 0.7849 - top_k_categorical_accuracy: 0.9684 - val_loss: 0.8191 - val_acc: 0.7681 - val_top_k_categorical_accuracy: 0.9598
Epoch 4/30
210012/210012 [==============================] - 9s 41us/step - loss: 0.6962 - acc: 0.7945 - top_k_categorical_accuracy: 0.9736 - val_loss: 0.7963 - val_acc: 0.7761 - val_top_k_categorical_accuracy: 0.9611
Epoch 5/30
210012/210012 [==============================] - 9s 45us/step - loss:

KeyboardInterrupt: 

In [64]:
def top_n_accuracy(preds,real,n):
    hit=0
    preds=[pred[-n:] for pred in preds.argsort()]
    real=[r[-1]for r in np.array(real).argsort()]
    for index,value in tqdm.tqdm(enumerate(real)):
        if(value in preds[index]):
            hit+=1
    return (hit/len(real))

In [84]:
def top_n_feat(preds,n):
    topn=[pred[-n:] for pred in preds.argsort()]
    new_feat=[]
    for value in tqdm.tqdm(preds):
        temp=[]
        for index,val in enumerate(value):
            if(index in topn[index]):
                temp.append(val)
            else:
                temp.append(0)
        new_feat.append(temp)
    return new_feat

In [ ]:
def top_n_data_bin(f,l,n):
    topn=[pred[-n:] for pred in preds.argsort()]
    new_feat=[]
    for value in tqdm.tqdm(preds):
        temp=[]
        for index,val in enumerate(value):
            if(index in topn[index]):
                temp.append(val)
            else:
                temp.append(0)
        new_feat.append(temp)
    return new_feat

In [77]:
abab=model.predict(nn_X_train)

In [71]:
baba=model.predict(nn_X_test)

In [74]:
tnacc=top_n_accuracy(baba,nn_y_test,5)


0it [00:00, ?it/s]
7133it [00:00, 70627.55it/s]
17854it [00:00, 88788.18it/s]
23335it [00:00, 92847.15it/s]

In [117]:
abab=model.predict_proba(nn_X_train)

In [118]:
nefeat=top_n_feat(abab,20)



  0%|          | 0/210012 [00:00<?, ?it/s]

  0%|          | 129/210012 [00:00<02:44, 1274.37it/s]

  0%|          | 252/210012 [00:00<02:47, 1249.81it/s]

  0%|          | 422/210012 [00:00<02:29, 1398.98it/s]

  0%|          | 636/210012 [00:00<02:12, 1582.78it/s]

  0%|          | 823/210012 [00:00<02:07, 1637.90it/s]

  0%|          | 1041/210012 [00:00<02:00, 1727.91it/s]

  1%|          | 1262/210012 [00:00<01:56, 1795.77it/s]

  1%|          | 1447/210012 [00:00<01:56, 1796.70it/s]

  1%|          | 1670/210012 [00:00<01:53, 1843.49it/s]

  1%|          | 1890/210012 [00:01<01:50, 1878.67it/s]

  1%|          | 2104/210012 [00:01<01:49, 1901.87it/s]

  1%|          | 2310/210012 [00:01<01:49, 1903.99it/s]

  1%|          | 2533/210012 [00:01<01:47, 1928.09it/s]

  1%|▏         | 2750/210012 [00:01<01:46, 1944.23it/s]

  1%|▏         | 2961/210012 [00:01<01:46, 1936.28it/s]

  2%|▏         | 3185/210012 [00:01<01:45, 1954.53it/s]

  2%|▏         | 3408/210012 [00:01<01:44, 1970

 23%|██▎       | 48699/210012 [00:28<01:35, 1692.96it/s]

 23%|██▎       | 48822/210012 [00:28<01:35, 1690.90it/s]

 23%|██▎       | 48942/210012 [00:28<01:35, 1688.99it/s]

 23%|██▎       | 49061/210012 [00:29<01:35, 1687.05it/s]

 23%|██▎       | 49179/210012 [00:29<01:35, 1684.79it/s]

 23%|██▎       | 49294/210012 [00:29<01:35, 1682.93it/s]

 24%|██▎       | 49409/210012 [00:29<01:35, 1679.56it/s]

 24%|██▎       | 49517/210012 [00:29<01:35, 1676.44it/s]

 24%|██▎       | 49620/210012 [00:29<01:35, 1673.48it/s]

 24%|██▎       | 49719/210012 [00:29<01:35, 1670.47it/s]

 24%|██▎       | 49815/210012 [00:29<01:36, 1667.73it/s]

 24%|██▍       | 49909/210012 [00:29<01:36, 1665.05it/s]

 24%|██▍       | 50010/210012 [00:30<01:36, 1662.84it/s]

 24%|██▍       | 50109/210012 [00:30<01:36, 1660.61it/s]

 24%|██▍       | 50218/210012 [00:30<01:36, 1658.70it/s]

 24%|██▍       | 50318/210012 [00:30<01:36, 1655.03it/s]

 24%|██▍       | 50507/210012 [00:30<01:36, 1655.79it/s]

 24%|██▍      

 43%|████▎     | 89836/210012 [00:56<01:15, 1589.43it/s]

 43%|████▎     | 90047/210012 [00:56<01:15, 1590.35it/s]

 43%|████▎     | 90250/210012 [00:56<01:15, 1591.05it/s]

 43%|████▎     | 90453/210012 [00:56<01:15, 1591.82it/s]

 43%|████▎     | 90672/210012 [00:56<01:14, 1592.87it/s]

 43%|████▎     | 90879/210012 [00:57<01:14, 1593.62it/s]

 43%|████▎     | 91085/210012 [00:57<01:14, 1594.33it/s]

 43%|████▎     | 91289/210012 [00:57<01:14, 1594.17it/s]

 44%|████▎     | 91497/210012 [00:57<01:14, 1595.01it/s]

 44%|████▎     | 91716/210012 [00:57<01:14, 1596.04it/s]

 44%|████▍     | 91942/210012 [00:57<01:13, 1597.19it/s]

 44%|████▍     | 92156/210012 [00:57<01:13, 1598.13it/s]

 44%|████▍     | 92383/210012 [00:57<01:13, 1599.29it/s]

 44%|████▍     | 92599/210012 [00:57<01:13, 1600.04it/s]

 44%|████▍     | 92815/210012 [00:57<01:13, 1600.97it/s]

 44%|████▍     | 93027/210012 [00:58<01:13, 1601.23it/s]

 44%|████▍     | 93244/210012 [00:58<01:12, 1602.19it/s]

 44%|████▍    

 67%|██████▋   | 140304/210012 [01:24<00:41, 1669.80it/s]

 67%|██████▋   | 140482/210012 [01:24<00:41, 1669.94it/s]

 67%|██████▋   | 140701/210012 [01:24<00:41, 1670.55it/s]

 67%|██████▋   | 140911/210012 [01:24<00:41, 1671.05it/s]

 67%|██████▋   | 141131/210012 [01:24<00:41, 1671.67it/s]

 67%|██████▋   | 141331/210012 [01:24<00:41, 1671.77it/s]

 67%|██████▋   | 141524/210012 [01:24<00:40, 1671.99it/s]

 67%|██████▋   | 141717/210012 [01:24<00:40, 1672.05it/s]

 68%|██████▊   | 141902/210012 [01:24<00:40, 1671.94it/s]

 68%|██████▊   | 142106/210012 [01:24<00:40, 1672.34it/s]

 68%|██████▊   | 142291/210012 [01:25<00:40, 1671.37it/s]

 68%|██████▊   | 142455/210012 [01:25<00:40, 1671.32it/s]

 68%|██████▊   | 142619/210012 [01:25<00:40, 1671.11it/s]

 68%|██████▊   | 142783/210012 [01:25<00:40, 1671.07it/s]

 68%|██████▊   | 142944/210012 [01:25<00:40, 1670.96it/s]

 68%|██████▊   | 143105/210012 [01:25<00:40, 1670.81it/s]

 68%|██████▊   | 143264/210012 [01:25<00:39, 1670.55it/s

 91%|█████████ | 191591/210012 [01:51<00:10, 1714.45it/s]

 91%|█████████▏| 191774/210012 [01:51<00:10, 1714.52it/s]

 91%|█████████▏| 191956/210012 [01:51<00:10, 1714.42it/s]

 91%|█████████▏| 192132/210012 [01:52<00:10, 1714.07it/s]

 92%|█████████▏| 192298/210012 [01:52<00:10, 1713.57it/s]

 92%|█████████▏| 192452/210012 [01:52<00:10, 1713.24it/s]

 92%|█████████▏| 192657/210012 [01:52<00:10, 1713.54it/s]

 92%|█████████▏| 192823/210012 [01:52<00:10, 1713.12it/s]

 92%|█████████▏| 193001/210012 [01:52<00:09, 1713.17it/s]

 92%|█████████▏| 193164/210012 [01:52<00:09, 1712.82it/s]

 92%|█████████▏| 193369/210012 [01:52<00:09, 1713.12it/s]

 92%|█████████▏| 193539/210012 [01:52<00:09, 1712.84it/s]

 92%|█████████▏| 193755/210012 [01:53<00:09, 1713.24it/s]

 92%|█████████▏| 193971/210012 [01:53<00:09, 1713.64it/s]

 92%|█████████▏| 194161/210012 [01:53<00:09, 1712.86it/s]

 93%|█████████▎| 194381/210012 [01:53<00:09, 1713.29it/s]

 93%|█████████▎| 194565/210012 [01:53<00:09, 1712.59it/s

In [119]:
nnn_X_train,nnn_X_test,nnn_y_train,nnn_y_test=train_test_split(model.predict(nn_X_train),nn_y_train,test_size=0.1)

In [120]:
mmodel = Sequential()
mmodel.add(Dense(500, input_shape=(110,), activation='relu'))
mmodel.add(Dense(750, activation='relu'))
mmodel.add(Dense(110, activation='softmax'))


mmodel.compile(optimizer="Adagrad", loss='categorical_crossentropy',metrics=['accuracy'])

In [121]:
history = mmodel.fit([nnn_X_train], [nnn_y_train], initial_epoch=0, epochs=25, batch_size=100, validation_data=([nnn_X_test],[nnn_y_test]), shuffle=True)

Train on 189010 samples, validate on 21002 samples
Epoch 1/25
189010/189010 [==============================] - 8s 44us/step - loss: 0.6393 - acc: 0.8358 - val_loss: 0.6036 - val_acc: 0.8366
Epoch 2/25
189010/189010 [==============================] - 8s 40us/step - loss: 0.5741 - acc: 0.8416 - val_loss: 0.5894 - val_acc: 0.8407
Epoch 3/25
189010/189010 [==============================] - 7s 39us/step - loss: 0.5628 - acc: 0.8419 - val_loss: 0.5813 - val_acc: 0.8385
Epoch 4/25
189010/189010 [==============================] - 7s 38us/step - loss: 0.5546 - acc: 0.8420 - val_loss: 0.5734 - val_acc: 0.8384
Epoch 5/25
189010/189010 [==============================] - 8s 40us/step - loss: 0.5485 - acc: 0.8418 - val_loss: 0.5712 - val_acc: 0.8382
Epoch 6/25
189010/189010 [==============================] - 7s 37us/step - loss: 0.5431 - acc: 0.8418 - val_loss: 0.5657 - val_acc: 0.8394
Epoch 7/25
189010/189010 [==============================] - 7s 38us/step - loss: 0.5385 - acc: 0.8425 - val_loss: 0